In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


In [2]:
from keras.models import load_model

In [3]:
model = load_model('/home/jordan/C4.h5')

In [14]:
from itertools import groupby, chain

NONE = 0
RED = 1
YELLOW = -1

def diagonalsPos (matrix, cols, rows):
    """Get positive diagonals, going from bottom-left to top-right."""
    for di in ([(j, i - j) for j in range(cols)] for i in range(cols + rows -1)):
        yield [matrix[i][j] for i, j in di if i >= 0 and j >= 0 and i < cols and j < rows]

def diagonalsNeg (matrix, cols, rows):
    """Get negative diagonals, going from top-left to bottom-right."""
    for di in ([(j, i - cols + j + 1) for j in range(cols)] for i in range(cols + rows - 1)):
        yield [matrix[i][j] for i, j in di if i >= 0 and j >= 0 and i < cols and j < rows]

class Game:
    def __init__ (self, cols = 7, rows = 6, requiredToWin = 4):
        """Create a new game."""
        self.cols = cols
        self.rows = rows
        self.win = requiredToWin
        self.board = [[NONE] * rows for _ in range(cols)]

    def insert (self, column, color):
        """Insert the color in the given column."""
        c = self.board[column]
        if c[0] != NONE:
            raise Exception('Column is full')
        i = -1
        while c[i] != NONE:
            i -= 1
        c[i] = color
        #self.checkForWin()
        dq,w = self.checkForWin()
        if dq == 'done':
            #print 'GAME OVER'            
            return 'end',w
        else:           
            return 'nd',-10    

    def checkForWin (self):
        """Check the current board for a winner."""
        w = self.getWinner()
        if w:
            #self.printBoard()
            #return w
            #self.printBoard()
            #raise Exception(str(w) + ' won!')
            return 'done',w
        else:
            return 'nd',-10

    def getWinner (self):
        """Get the winner on the current board."""
        lines = (
            self.board, # columns
            zip(*self.board), # rows
            diagonalsPos(self.board, self.cols, self.rows), # positive diagonals
            diagonalsNeg(self.board, self.cols, self.rows) # negative diagonals
        )

        for line in chain(*lines):
            for color, group in groupby(line):
                if color != NONE and len(list(group)) >= self.win:
                    return color

    def printBoard (self):
        """Print the board."""
        #print('  '.join(map(str, range(self.cols))))
        for y in range(self.rows):
            print('  '.join(str(self.board[x][y]) for x in range(self.cols)))
        #print()

In [185]:
g = Game()
turn = RED

# Game

In [341]:
g.printBoard()

0  0  0  0  0  1  0
1  0  -1  1  0  -1  -1
-1  0  1  -1  0  -1  1
1  0  -1  -1  0  -1  -1
1  0  1  1  0  1  1
1  0  -1  1  0  -1  -1


In [342]:
bc = np.copy(g.board)
boards = []
locs = np.zeros(7)
for i in range(7):
    flag = True
    g.board = np.copy(bc)
    board_tmp = np.transpose(g.board)
    locations_0 = len(np.where(board_tmp == 0)[0])
    top_0 = board_tmp[:,i][0]       
    if top_0 != 0:
        flag = False
    if locations_0 == 0:
        flag = False   
    if flag == True:
        status,winner = g.insert(int(i), turn)
        boards.append(np.transpose([g.board]))
        locs[i] = 1
    if flag == False: 
        boards.append(np.zeros((6,7,1)))

In [343]:
predicted = model.predict(np.array(boards)).flatten()

In [344]:
predicted

array([0.5829842 , 0.5927363 , 0.5115231 , 0.6258294 , 0.53786623,
       0.58839303, 0.6188033 ], dtype=float32)

In [345]:
g.board = np.copy(bc)

In [346]:
g.printBoard()

0  0  0  0  0  1  0
1  0  -1  1  0  -1  -1
-1  0  1  -1  0  -1  1
1  0  -1  -1  0  -1  -1
1  0  1  1  0  1  1
1  0  -1  1  0  -1  -1


In [347]:
row = np.argmax(predicted)

In [348]:
g.insert(int(row), turn)

('nd', -10)

In [349]:
g.printBoard()

0  0  0  1  0  1  0
1  0  -1  1  0  -1  -1
-1  0  1  -1  0  -1  1
1  0  -1  -1  0  -1  -1
1  0  1  1  0  1  1
1  0  -1  1  0  -1  -1


In [338]:
turn = YELLOW if turn == RED else RED

In [339]:
g.insert(int(6), turn)
turn = YELLOW if turn == RED else RED

In [340]:
g.printBoard()

0  0  0  0  0  1  0
1  0  -1  1  0  -1  -1
-1  0  1  -1  0  -1  1
1  0  -1  -1  0  -1  -1
1  0  1  1  0  1  1
1  0  -1  1  0  -1  -1
